In [1]:
import os
import pandas as pd
import numpy as np
import chess
import chess.pgn
# Import các lớp cần thiết từ ChessEngine.py
from ChessEngine import GameState, Move

# --- Hàm trích xuất đặc trưng (Đã điều chỉnh cho ChessEngine) ---

def get_board_features(game_state):
    """Lấy đặc trưng bàn cờ từ đối tượng GameState."""
    board_features = []
    for r in range(8):
        for c in range(8):
            # Lấy quân cờ theo định dạng của ChessEngine ('wp', 'bR', '--', etc.)
            piece = game_state.board[r][c]
            board_features.append(piece)
    # Trả về list 64 phần tử ['bR', 'bN', ..., '--', ..., 'wp', 'wR']
    return board_features

def get_move_features(engine_move):
    """Lấy đặc trưng nước đi từ đối tượng ChessEngine.Move."""
    from_ = np.zeros(64)
    to_ = np.zeros(64)
    # Tính toán chỉ số phẳng (0-63) từ tọa độ hàng/cột (0-7)
    # Lưu ý: ChessEngine dùng (hàng, cột) với hàng 0 là hàng trên cùng (quân đen)
    # Chỉ số 0-63 thường bắt đầu từ a1 (hàng 7, cột 0 trong ChessEngine)
    # Do đó, cần ánh xạ cẩn thận hoặc đảm bảo tính nhất quán
    # Cách ánh xạ phổ biến: index = row * 8 + col
    # Ví dụ: (0,0) -> 0, (0,7) -> 7, (7,0) -> 56, (7,7) -> 63
    from_idx = engine_move.start_row * 8 + engine_move.start_col
    to_idx = engine_move.end_row * 8 + engine_move.end_col
    from_[from_idx] = 1
    to_[to_idx] = 1
    return from_, to_

# --- Hàm chuyển đổi nước đi ---

def find_engine_move(pgn_chess_move, valid_engine_moves):
    """
    Tìm đối tượng ChessEngine.Move tương ứng với chess.Move từ PGN
    trong danh sách các nước đi hợp lệ của ChessEngine.
    """
    # Chuyển đổi tọa độ từ chess.Move (0-63, a1=0) sang ChessEngine (row, col, 0-7)
    start_col = pgn_chess_move.from_square % 8
    # Hàng trong python-chess (0=a1..7=a8) -> Hàng trong ChessEngine (0=rank8..7=rank1)
    start_row = 7 - (pgn_chess_move.from_square // 8)
    end_col = pgn_chess_move.to_square % 8
    end_row = 7 - (pgn_chess_move.to_square // 8)

    # Xử lý phong cấp (Promotion)
    # chess.Move có thuộc tính 'promotion' (ví dụ: chess.QUEEN)
    # ChessEngine.Move tự động xác định phong cấp trong constructor nếu là tốt đến hàng cuối
    # Nhưng ta cần đảm bảo chọn đúng nước đi phong cấp nếu có nhiều lựa chọn (hiếm)
    # Hiện tại, giả định phong cấp mặc định của ChessEngine (thành Hậu) là đủ
    # Hoặc bạn có thể thêm logic kiểm tra promotion piece nếu cần

    for engine_move in valid_engine_moves:
        if (engine_move.start_row == start_row and
            engine_move.start_col == start_col and
            engine_move.end_row == end_row and
            engine_move.end_col == end_col):
            # Nếu là phong cấp, cần kiểm tra thêm loại quân phong cấp (nếu PGN có thông tin)
            # Ví dụ: if pgn_chess_move.promotion is not None: ...
            # Hiện tại chỉ khớp tọa độ là đủ trong hầu hết các trường hợp
            return engine_move # Trả về đối tượng Move từ ChessEngine

    # Không tìm thấy nước đi khớp -> có thể là lỗi PGN hoặc khác biệt logic
    # print(f"Cảnh báo: Không tìm thấy Engine move khớp với PGN move {pgn_chess_move.uci()}")
    # print(f"   Tọa độ mong muốn: ({start_row},{start_col}) -> ({end_row},{end_col})")
    # print(f"   Các engine moves hợp lệ bắt đầu từ ({start_row},{start_col}):")
    # for m in valid_engine_moves:
    #     if m.start_row == start_row and m.start_col == start_col:
    #         print(f"      -> ({m.end_row},{m.end_col})")
    return None

# --- Hàm xử lý chính (Đã sửa đổi) ---

def process_game_data(dirname, filename):
    game_path = os.path.join(dirname, filename)
    pgn = None
    game = None

    try:
        pgn = open(game_path, encoding='utf-8')
        headers = chess.pgn.read_headers(pgn)
        if headers is None:
            # print(f"Không thể đọc headers từ file: {filename}. Bỏ qua.")
            return

        result = headers.get('Result', '*')
        if result == "*":
            return

        result_mapping = {'1-0': True, '1/2-1/2': None, '0-1': False}
        result_value = result_mapping.get(result, None)
        if result_value is None:
            return

        # Đọc lại toàn bộ game
        pgn.seek(0) # Quay lại đầu file
        game = chess.pgn.read_game(pgn)
        if game is None:
            # print(f"Không thể đọc toàn bộ game từ file: {filename}. Bỏ qua.")
            return

    except Exception as e:
        # print(f"Lỗi khi đọc file PGN {filename}: {e}. Bỏ qua.")
        return
    finally:
        if pgn:
            pgn.close()

    # Xác định bên thắng
    white_won = result_value

    # Khởi tạo GameState từ ChessEngine
    game_state = GameState()
    local_data = [] # Dữ liệu chỉ cho game này

    try:
        node = game # Bắt đầu từ node gốc của game PGN
        # Lặp qua các nước đi trong PGN
        while not node.is_end():
            next_node = node.variation(0) # Lấy nước đi tiếp theo trong dòng chính
            pgn_move = next_node.move # Đây là đối tượng chess.Move

            # Kiểm tra lượt đi hiện tại của ChessEngine
            is_winning_side_turn = (white_won and game_state.white_to_move) or \
                                   (not white_won and not game_state.white_to_move)

            engine_move_to_make = None

            if is_winning_side_turn:
                # Lấy các nước đi hợp lệ từ ChessEngine
                valid_engine_moves = game_state.getValidMoves()

                # Tìm nước đi ChessEngine tương ứng với nước đi PGN
                good_engine_move = find_engine_move(pgn_move, valid_engine_moves)

                if good_engine_move is None:
                    print(f"Cảnh báo: Không tìm thấy nước đi Engine hợp lệ khớp với PGN move {pgn_move.uci()} trong file {filename}. Bỏ qua game.")
                    return # Bỏ qua game này nếu có nước đi không khớp

                engine_move_to_make = good_engine_move

                # Lấy các nước đi "xấu" (các nước đi hợp lệ khác)
                bad_engine_moves = list(filter(lambda m: m != good_engine_move, valid_engine_moves))

                # Lấy đặc trưng bàn cờ HIỆN TẠI (trước khi thực hiện nước đi)
                board_features = get_board_features(game_state)

                # Thêm dữ liệu cho các nước đi xấu
                for bad_move in bad_engine_moves:
                    from_feat, to_feat = get_move_features(bad_move)
                    line = np.concatenate((board_features, from_feat, to_feat, [False]))
                    local_data.append(line)

                # Thêm dữ liệu cho nước đi tốt
                from_feat, to_feat = get_move_features(good_engine_move)
                line = np.concatenate((board_features, from_feat, to_feat, [True]))
                local_data.append(line)

            else:
                # Nếu không phải lượt của bên thắng, vẫn cần tìm và thực hiện nước đi
                # để chuyển trạng thái bàn cờ cho đúng
                valid_engine_moves = game_state.getValidMoves()
                engine_move_found = find_engine_move(pgn_move, valid_engine_moves)

                if engine_move_found is None:
                    print(f"Cảnh báo: (Lượt bên thua) Không tìm thấy nước đi Engine hợp lệ khớp với PGN move {pgn_move.uci()} trong file {filename}. Bỏ qua game.")
                    return # Bỏ qua game này

                engine_move_to_make = engine_move_found

            # Thực hiện nước đi trên GameState của ChessEngine
            if engine_move_to_make:
                 # Kiểm tra lại xem nước đi có thực sự trong valid_moves không
                 # (find_engine_move đã làm điều này, nhưng kiểm tra lại cho chắc)
                 current_valid_moves = game_state.getValidMoves()
                 is_truly_valid = any(m == engine_move_to_make for m in current_valid_moves)

                 if is_truly_valid:
                     game_state.makeMove(engine_move_to_make)
                 else:
                    # Điều này không nên xảy ra nếu find_engine_move hoạt động đúng
                    print(f"LỖI NGHIÊM TRỌNG: Nước đi {engine_move_to_make} được chọn nhưng không hợp lệ trong {filename}!")
                    return # Dừng xử lý game lỗi
            else:
                # Điều này cũng không nên xảy ra nếu logic trên đúng
                print(f"LỖI LOGIC: Không xác định được engine_move_to_make trong {filename}.")
                return

            # Chuyển sang node PGN tiếp theo
            node = next_node

    except ValueError as e:
        print(f"Lỗi ValueError khi xử lý nước đi trong file {filename}: {e}. Bỏ qua file.")
        return
    except Exception as e:
        print(f"Lỗi không xác định khi xử lý game {filename}: {e}. Bỏ qua file.")
        return

    # --- Lưu dữ liệu nếu có ---
    if not local_data:
        # print(f"Không có dữ liệu được tạo ra cho game {filename}. Bỏ qua.")
        return

    # Định nghĩa tên cột (vẫn có thể dùng SQUARE_NAMES cho tiện)
    board_feature_names = chess.SQUARE_NAMES
    move_from_feature_names = ['from_' + square for square in chess.SQUARE_NAMES]
    move_to_feature_names = ['to_' + square for square in chess.SQUARE_NAMES]
    columns = board_feature_names + move_from_feature_names + move_to_feature_names + ['good_move']

    try:
        df = pd.DataFrame(data=local_data, columns=columns)
        # print(f"Processed {filename}: {df.shape}")

        # Tạo thư mục lưu nếu chưa có
        new_dirname = 'D:/AI/Chess/ChessApp/Chess/csv_data/abdusattorov' # Thư mục đích mới
        os.makedirs(new_dirname, exist_ok=True)

        new_filename = filename.replace('.pgn', '.csv')
        new_path = os.path.join(new_dirname, new_filename)

        df.to_csv(new_path, index=False)

    except Exception as e:
        print(f"Lỗi khi tạo DataFrame hoặc lưu CSV cho file {filename}: {e}")


# --- Vòng lặp chính ---
source_directory = 'D:/AI/Chess/ChessApp/Chess/pgn_data/abdusattorov' # Thư mục chứa file PGN
print(f"Bắt đầu xử lý các file PGN từ: {source_directory} bằng ChessEngine logic")
file_count = 0
processed_count = 0
error_count = 0

# Đảm bảo đường dẫn tồn tại
if not os.path.isdir(source_directory):
    print(f"LỖI: Thư mục nguồn không tồn tại: {source_directory}")
else:
    for dirname, _, filenames in os.walk(source_directory):
        for filename in filenames:
            if filename.lower().endswith('.pgn'):
                file_count += 1
                # print(f"Processing file ({file_count}): {filename}...")
                try:
                    process_game_data(dirname, filename)
                    processed_count +=1
                except Exception as e:
                    print(f"!!!!!!!! Lỗi không mong muốn khi xử lý file {filename}: {e} !!!!!!!!!!")
                    error_count += 1

    print("\n--- Hoàn thành xử lý ---")
    print(f"Tổng số file PGN tìm thấy: {file_count}")
    print(f"Số file đã xử lý (hoặc bỏ qua hợp lệ): {processed_count}") # Đã xử lý không có nghĩa là đã tạo CSV
    print(f"Số file gây lỗi và bị bỏ qua hoàn toàn: {error_count}")

Bắt đầu xử lý các file PGN từ: D:/AI/Chess/ChessApp/Chess/pgn_data/abdusattorov bằng ChessEngine logic
Cảnh báo: Không tìm thấy nước đi Engine hợp lệ khớp với PGN move h4g3 trong file game_1036.pgn. Bỏ qua game.
Cảnh báo: Không tìm thấy nước đi Engine hợp lệ khớp với PGN move g5g4 trong file game_1092.pgn. Bỏ qua game.
Cảnh báo: (Lượt bên thua) Không tìm thấy nước đi Engine hợp lệ khớp với PGN move h7h8 trong file game_1415.pgn. Bỏ qua game.
Cảnh báo: Không tìm thấy nước đi Engine hợp lệ khớp với PGN move b3b2 trong file game_1507.pgn. Bỏ qua game.
Cảnh báo: (Lượt bên thua) Không tìm thấy nước đi Engine hợp lệ khớp với PGN move f5g6 trong file game_1537.pgn. Bỏ qua game.
Cảnh báo: Không tìm thấy nước đi Engine hợp lệ khớp với PGN move h4h5 trong file game_2095.pgn. Bỏ qua game.
Cảnh báo: (Lượt bên thua) Không tìm thấy nước đi Engine hợp lệ khớp với PGN move g2g3 trong file game_2131.pgn. Bỏ qua game.
Cảnh báo: Không tìm thấy nước đi Engine hợp lệ khớp với PGN move h4h5 trong file game_